## Q1.

Add methods `__iter__` to your project Time Series class to iterate over values, a method `itertimes` to iterate over times, a method `itervalues` to iterate over values, and a method `iteritems` to iterate over time-value pairs. (This is a similar interface to python dictionaries). To test these, check both the types of the results and the answers you expect.

In [1]:
#your code here


## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [14]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [15]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [16]:
g = make_data(5, 10)
list(g)

[999999999.6949916,
 999999999.8731143,
 1000000000.874608,
 999999998.2258589,
 999999999.984858,
 999999997.9674028,
 1000000000.3574641,
 999999998.2571396,
 1000000000.67899,
 1000000000.3744192,
 999999999.636875]

In [17]:
g = online_mean(make_data(5, 100))
print(type(g))
list(g)

<class 'generator'>


[999999996.733125,
 999999997.3625338,
 999999997.7538651,
 999999998.8259468,
 999999999.4049274,
 999999999.6505694,
 999999999.986985,
 1000000000.1195996,
 1000000000.0281813,
 1000000000.0501871,
 1000000000.2436261,
 1000000000.222019,
 1000000000.2033015,
 1000000000.409862,
 1000000000.4252728,
 1000000000.3151098,
 1000000000.4013369,
 1000000000.3813795,
 1000000000.1711766,
 1000000000.4527487,
 1000000000.3699626,
 1000000000.1718608,
 999999999.7569395,
 999999999.8536801,
 999999999.8366659,
 1000000000.0935988,
 999999999.7104682,
 999999999.741009,
 999999999.6156029,
 1000000000.0574178,
 1000000000.0338672,
 1000000000.1099402,
 1000000000.0318493,
 1000000000.0000029,
 999999999.9709872,
 1000000000.0833205,
 1000000000.0847766,
 1000000000.1429358,
 1000000000.0820522,
 1000000000.1613166,
 1000000000.0694326,
 1000000000.1717634,
 1000000000.1700509,
 1000000000.1591498,
 1000000000.0833992,
 1000000000.0682601,
 1000000000.1218482,
 1000000000.1545197,
 1000000000

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [18]:
import math
# your code here
def online_mean_dev(iterator):
    n = 0
    for value in iterator:
        n = n+1
        if n==1:
            mu = value
            old_mu = value
            old_dev_accum = 0.0
        elif n > 1:
            mu = old_mu + 1.0*(value - old_mu)/n
            dev_accum = old_dev_accum + (value - old_mu)*(value - mu)
            stddev = math.sqrt(dev_accum/(n-1))
            yield (n, value, mu, stddev)
            old_mu = mu
            old_dev_accum = dev_accum

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [36]:
data_with_stats = online_mean_dev(make_data(5, 100000))

In [20]:
data_with_stats = online_mean_dev(make_data(5, 10))
for i in data_with_stats:
    print (i)

(2, 999999997.5948428, 999999999.3371397, 2.4639799539246083)
(3, 1000000001.828266, 1000000000.1675152, 2.259240732127109)
(4, 1000000000.9333627, 1000000000.3589771, 1.883987722602373)
(5, 1000000000.0462463, 1000000000.296431, 1.6375644767007431)
(6, 999999999.1924468, 1000000000.1124336, 1.5324568766503766)
(7, 999999998.9926617, 999999999.9524661, 1.461556382128187)
(8, 1000000000.0239923, 999999999.961407, 1.35337455693341)
(9, 1000000000.2054247, 999999999.98852, 1.2685763386218456)
(10, 999999996.715831, 999999999.6612511, 1.581621217919134)
(11, 1000000000.2731227, 999999999.7168758, 1.5117566854253919)


## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.

In [37]:
#your code here
def is_ok(level, t):
    n, value, mu, stddev = t
    if (mu - level*stddev <= value) and (value <= mu + level*stddev):
        return True
    else:
        return False

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [38]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [39]:
list(anomalies)#materialize

[(5499, 1000000015.7011416, 999999999.9811611, 2.9290150856888975),
 (12319, 999999983.3022345, 1000000000.0105721, 2.8772237781317958),
 (15557, 1000000015.9418716, 999999999.9905121, 2.8577782885846164),
 (19714, 1000000014.7616296, 999999999.9957602, 2.8621442970405955),
 (24026, 1000000014.5878202, 999999999.9962343, 2.8864668216983307),
 (24760, 999999983.4624264, 999999999.9982487, 2.8779812892024523),
 (27507, 999999985.3455403, 999999999.9948539, 2.885204672398216),
 (28133, 1000000014.7055862, 999999999.999545, 2.8890541657569253),
 (31923, 999999985.0781586, 999999999.9965832, 2.881243805589433),
 (33295, 999999985.460763, 999999999.9969776, 2.8822506386938165),
 (34456, 1000000017.7530051, 999999999.9972781, 2.8820552572577647),
 (47616, 1000000016.0321847, 999999999.9935539, 2.8906634074489848),
 (47892, 999999985.0810857, 999999999.9936116, 2.892584770419964),
 (48748, 1000000015.7547781, 999999999.9938732, 2.891397982386108),
 (49190, 999999985.3514779, 999999999.9911772,

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).